In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd

In [ ]:
#Move indicator into its own Class so that will show up in output file. 
#Reference: https://www.backtrader.com/docu/inddev.html
class Bollinger_Breakout(bt.Indicator):
    lines = ('breakout',)
    
    def __init__(self):
        self.lines.breakout = self.data > bt.indicators.BBands(self.data, period=20).lines.top

In [146]:
class Bollinger_Breakdown(bt.Indicator):
    lines = ('breakdown',)
    
    def __init__(self):
        self.lines.breakdown = self.data < bt.indicators.BBands(self.data, period=20).lines.bot

In [147]:
class Bollinger_Upperbelt(bt.Indicator):
    lines = ('upperbelt',)
    
    def __init__(self):
        self.lines.upperbelt = self.data > bt.indicators.BBands(self.data, period=20).lines.mid

In [148]:
class Bollinger_Lowerbelt(bt.Indicator):
    lines = ('lowerbelt',)
    
    def __init__(self):
        self.lines.lowerbelt = self.data < bt.indicators.BBands(self.data, period=20).lines.mid

In [219]:
'''
class Bollinger_Opening(bt.Indicator):
    lines = ('Opening',)
    
    def __init__(self):
        if ((bt.indicators.BBands(self.data, period=20).lines.top - bt.indicators.BBands(self.data, period=20).lines.bot)/bt.indicators.BBands(self.data, period=20).lines.mid) < 0.1:
            self.lines.opening = self.data
'''

"\nclass Bollinger_Opening(bt.Indicator):\n    lines = ('Opening',)\n    \n    def __init__(self):\n        if ((bt.indicators.BBands(self.data, period=20).lines.top - bt.indicators.BBands(self.data, period=20).lines.bot)/bt.indicators.BBands(self.data, period=20).lines.mid) < 0.1:\n            self.lines.opening = self.data\n"

In [220]:
# Create a Stratey to trade hv
class HVStrategy(bt.Strategy):
    params = (
        ("symbols", None),
        ("max_days_in_trade", 7),
        ("y", None)
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
      
        # Keep a reference to the close price, historical vol as well as returns
        self.stock_px = self.datas[symbols.index("stockpx")].close
        self.vol = self.datas[symbols.index(y)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
#         self.sma_short = bt.indicators.SimpleMovingAverage(self.stock_px, period=50)
#         self.sma_long = bt.indicators.SimpleMovingAverage(self.stock_px, period=200)
#         self.dsma = self.sma_short - self.sma_long
#         self.mom = bt.Cmp(self.sma_short, self.sma_long)
        
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
        self.bollinger_breakout = Bollinger_Breakout(self.stock_px)
        self.bollinger_breakdown = Bollinger_Breakdown(self.stock_px)
        self.bollinger_upperbelt = Bollinger_Upperbelt(self.stock_px)
        self.bollinger_lowerbelt = Bollinger_Lowerbelt(self.stock_px)
        
        #self.bollinger_opening = Bollinger_Opening(self.stock_px)
            
#         self.rsi_oversold = RSI_Oversold(self.stock_px)
#         self.rsi_overbought = RSI_Overbought(self.stock_px)
#        self.rsi_oversold = bt.indicators.RSI(self.stock_px) < 30
#        self.rsi_overbought = bt.indicators.RSI(self.stock_px) > 70

#         self.cross_up = bt.indicators.CrossUp(self.sma_short, self.sma_long) 
#         self.cross_down = bt.indicators.CrossDown(self.sma_short, self.sma_long)

        self.in_trade = False
        self.days_in_trade = 0
        self.flag = 0   # -1 sell, 0 no pos, 1 buy.
        
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f; Vol, %.2f' % (self.stock_px[0], self.vol[0]))
        
        size = self.position.size

        # If not in the trade
        if (self.flag == 0):
            if (self.bollinger_upperbelt[-3] > 0 and self.bollinger_upperbelt[-2] and self.bollinger_upperbelt[-1] > 0 and self.bollinger_breakout[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
                print("Breakout indicator triggered...entering position")
                self.buy(exectype=bt.Order.Close)
                self.days_in_trade = 1
                self.flag = 1
                
            '''
            if (self.bollinger_upperbelt[-3] > 0 and self.bollinger_upperbelt[-2] and self.bollinger_upperbelt[-1] > 0  and self.bollinger_breakdown[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
                print("111Breakdown indicator triggered...entering position")
                self.buy(exectype=bt.Order.Close)
                self.days_in_trade = 1
                self.flag = 1
            
            if (self.bollinger_lowerbelt[-3] > 0 and self.bollinger_lowerbelt[-2] > 0 and self.bollinger_lowerbelt[-1] > 0 and self.bollinger_breakout[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
                print("111Breakout indicator triggered...entering position")
                self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
                self.flag = -1
            '''
            if (self.bollinger_lowerbelt[-3] > 0 and self.bollinger_lowerbelt[-2] > 0 and self.bollinger_lowerbelt[-1] > 0 and self.bollinger_breakdown[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
                print("Breakdown indicator triggered...entering position")
                self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
                self.flag = -1
            
        # If in the long position
        elif (self.flag == 1):
            # Close the position if the SELL signal triggered or reached the max days to hold
            if ((self.bollinger_breakout[0] > 0 and self.bollinger_lowerbelt[0] > 0) or self.days_in_trade == self.p.max_days_in_trade):
                #self.bollinger_upperbelt[3] > 0 and self.bollinger_upperbelt[2] and self.bollinger_upperbelt[1] > 0 Holding for three days?
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
                self.flag = 0
                    
            elif (self.bollinger_breakdown[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
                self.days_in_trade += 1
                print("In position %d days" % self.days_in_trade)
        
        # If in the short position
        elif (self.flag == -1):
            # Close the position if the BUY signal triggered or reached the max days to hold
            if ((self.bollinger_breakdown[0] > 0 and self.bollinger_upperbelt[0] > 0) or self.days_in_trade == self.p.max_days_in_trade):
                self.close(exectype=bt.Order.Close)
                print("Closing the position")
                self.days_in_trade = 0
                self.flag = 0
                    
            elif (self.bollinger_breakout[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
                self.days_in_trade += 1
                print("In position %d days" % self.days_in_trade)
                
#         print("Flag = %d" % self.flag)
#         print("In position %d days" % self.days_in_trade)
            
        
        # Check if we are currently in a trade
#         if(not size):
#             #If not has our indicator triggered?
#             if self.rsi_oversold[0] > 0:
#                 print("Oversold indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
                
#             if self.rsi_overbought[0] > 0:
#                 print("Overbought indicator triggered...entering position")
#                 self.sell(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
                               
#         else:
#             self.days_in_trade += 1
#             print("In position %d days" % self.days_in_trade)
            
#             if self.days_in_trade == self.p.max_days_in_trade:
#                 print("Max days to hold trade reached...closing position")
#                 self.close(exectype=bt.Order.Close)
#                 self.days_in_trade = 0

In [224]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    # Add data
    datapath = os.path.join('../../../datas/MSFT.csv')
    
    datas = pd.read_csv(datapath, parse_dates=True,index_col=0)
    
    y = '10dORHV' #iv30
    symbol_cols = [y,'stockpx'] #symbol we're trying to forecast/trade first
    #symbol_cols = list(datas) #Load all symbols
    
    
    # Add the Data Feeds to Cerebro
    for i, symbol in enumerate(symbol_cols):
        data = datas[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add a strategy
    cerebro.addstrategy(HVStrategy, symbols=symbol_cols, y=y)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='hv-strategy-MSFT-bollinger-NEW(Belts).csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2010-01-04, Close, 214.31; Vol, 20.07
2010-01-05, Close, 214.27; Vol, 19.46
2010-01-06, Close, 210.84; Vol, 19.81
2010-01-07, Close, 210.42; Vol, 19.77
2010-01-08, Close, 211.83; Vol, 19.92
2010-01-11, Close, 210.23; Vol, 18.77
2010-01-12, Close, 207.33; Vol, 18.67
2010-01-13, Close, 210.74; Vol, 19.64
2010-01-14, Close, 209.42; Vol, 18.69
2010-01-15, Close, 206.01; Vol, 19.51
2010-01-19, Close, 214.97; Vol, 21.50
2010-01-20, Close, 211.94; Vol, 22.69
2010-01-21, Close, 207.92; Vol, 23.22
2010-01-22, Close, 197.94; Vol, 25.74
2010-01-25, Close, 202.80; Vol, 28.31
2010-01-26, Close, 205.87; Vol, 32.33
2010-01-27, Close, 208.27; Vol, 33.60
2010-01-28, Close, 199.22; Vol, 33.78
2010-01-29, Close, 192.66; Vol, 35.66
2010-02-01, Close, 194.58; Vol, 35.21
2010-02-02, Close, 195.70; Vol, 34.38
2010-02-03, Close, 199.47; Vol, 34.43
2010-02-04, Close, 192.06; Vol, 34.76
2010-02-05, Close, 195.63; Vol, 33.52
2010-02-08, Close, 194.32; Vol, 31.45
2010-02-09, Cl

2011-09-27, Close, 399.93; Vol, 32.19
In position 5 days
2011-09-28, Close, 397.44; Vol, 32.07
In position 6 days
2011-09-29, Close, 389.79; Vol, 35.47
In position 7 days
2011-09-30, Close, 382.73; Vol, 35.83
2011-10-03, Close, 376.95; Vol, 34.22
2011-10-04, Close, 371.01; Vol, 42.28
2011-10-05, Close, 378.10; Vol, 44.09
2011-10-06, Close, 376.49; Vol, 43.46
2011-10-07, Close, 371.20; Vol, 43.91
2011-10-11, Close, 400.79; Vol, 43.31
2011-10-12, Close, 402.81; Vol, 44.90
2011-10-13, Close, 407.12; Vol, 42.39
2011-10-14, Close, 421.34; Vol, 42.81
2011-10-17, Close, 419.73; Vol, 42.62
2011-10-18, Close, 422.71; Vol, 33.32
2011-10-19, Close, 398.34; Vol, 37.26
2011-10-20, Close, 395.24; Vol, 34.91
2011-10-21, Close, 391.74; Vol, 34.96
2011-10-24, Close, 404.91; Vol, 34.44
2011-10-25, Close, 398.26; Vol, 34.26
2011-10-26, Close, 400.92; Vol, 33.57
2011-10-27, Close, 405.14; Vol, 35.93
2011-10-28, Close, 405.71; Vol, 34.92
2011-10-31, Close, 405.85; Vol, 34.76
2011-11-01, Close, 395.18; Vol,

2013-06-11, Close, 441.17; Vol, 22.21
2013-06-12, Close, 434.11; Vol, 22.53
2013-06-13, Close, 435.07; Vol, 22.27
2013-06-14, Close, 429.80; Vol, 22.38
2013-06-17, Close, 432.06; Vol, 21.05
2013-06-18, Close, 432.13; Vol, 20.27
2013-06-19, Close, 425.07; Vol, 20.43
Breakdown indicator triggered...entering position
2013-06-20, Close, 420.92; Vol, 20.55
Closing the position
2013-06-21, Close, 411.78; Vol, 21.59
Breakdown indicator triggered...entering position
2013-06-24, Close, 402.40; Vol, 22.05
Closing the position
2013-06-25, Close, 403.69; Vol, 22.15
Breakdown indicator triggered...entering position
2013-06-26, Close, 397.62; Vol, 21.87
Closing the position
2013-06-27, Close, 394.56; Vol, 22.28
2013-06-28, Close, 399.83; Vol, 24.66
2013-07-01, Close, 411.35; Vol, 26.08
2013-07-02, Close, 415.24; Vol, 27.26
2013-07-03, Close, 419.29; Vol, 27.80
2013-07-05, Close, 416.35; Vol, 27.30
2013-07-08, Close, 414.26; Vol, 27.23
2013-07-09, Close, 422.58; Vol, 27.61
2013-07-10, Close, 420.35; 

2015-03-04, Close, 128.62; Vol, 19.13
2015-03-05, Close, 126.29; Vol, 20.30
2015-03-06, Close, 126.67; Vol, 23.56
2015-03-09, Close, 127.36; Vol, 23.88
2015-03-10, Close, 125.04; Vol, 24.12
2015-03-11, Close, 122.42; Vol, 24.27
2015-03-12, Close, 124.47; Vol, 23.05
2015-03-13, Close, 123.12; Vol, 23.44
2015-03-16, Close, 124.66; Vol, 23.74
2015-03-17, Close, 127.18; Vol, 23.53
2015-03-18, Close, 128.65; Vol, 24.33
2015-03-19, Close, 127.88; Vol, 23.74
2015-03-20, Close, 127.80; Vol, 21.45
2015-03-23, Close, 127.17; Vol, 19.58
2015-03-24, Close, 126.89; Vol, 18.44
2015-03-25, Close, 123.65; Vol, 18.98
2015-03-26, Close, 124.33; Vol, 18.73
2015-03-27, Close, 123.08; Vol, 17.99
2015-03-30, Close, 126.11; Vol, 18.35
2015-03-31, Close, 124.44; Vol, 18.36
2015-04-01, Close, 124.07; Vol, 17.73
2015-04-02, Close, 125.54; Vol, 17.62
2015-04-06, Close, 127.44; Vol, 19.06
2015-04-07, Close, 126.21; Vol, 20.00
2015-04-08, Close, 125.39; Vol, 20.00
2015-04-09, Close, 126.26; Vol, 18.50
2015-04-10, 

2016-11-02, Close, 111.92; Vol, 23.18
In position 2 days
2016-11-03, Close, 109.67; Vol, 23.53
Closing the position
2016-11-04, Close, 109.19; Vol, 25.20
Breakdown indicator triggered...entering position
2016-11-07, Close, 109.95; Vol, 26.32
In position 2 days
2016-11-08, Close, 111.04; Vol, 26.80
In position 3 days
2016-11-09, Close, 111.09; Vol, 20.88
In position 4 days
2016-11-10, Close, 108.27; Vol, 26.60
In position 5 days
2016-11-14, Close, 105.64; Vol, 29.70
Closing the position
2016-11-15, Close, 107.34; Vol, 28.96
2016-11-16, Close, 109.95; Vol, 30.30
2016-11-17, Close, 109.72; Vol, 30.22
2016-11-18, Close, 110.00; Vol, 28.78
2016-11-21, Close, 111.80; Vol, 28.21
2016-11-22, Close, 111.72; Vol, 27.73
2016-11-23, Close, 110.94; Vol, 26.02
2016-11-25, Close, 111.79; Vol, 19.73
2016-11-28, Close, 111.76; Vol, 18.18
2016-11-29, Close, 111.08; Vol, 14.89
2016-11-30, Close, 110.63; Vol, 15.87
2016-12-01, Close, 109.76; Vol, 14.07
2016-12-02, Close, 109.64; Vol, 13.95
2016-12-05, Clo

2017-11-01, Close, 166.89; Vol, 23.58
In position 2 days
2017-11-02, Close, 168.11; Vol, 22.95
In position 3 days
2017-11-03, Close, 172.50; Vol, 24.60
2017-11-06, Close, 174.25; Vol, 25.08
Breakout indicator triggered...entering position
2017-11-07, Close, 174.81; Vol, 25.08
In position 2 days
2017-11-08, Close, 176.24; Vol, 25.00
In position 3 days
2017-11-09, Close, 175.88; Vol, 25.37
In position 4 days
2017-11-10, Close, 174.67; Vol, 21.25
In position 5 days
2017-11-13, Close, 173.97; Vol, 18.68
In position 6 days
2017-11-14, Close, 171.34; Vol, 18.59
In position 7 days
2017-11-17, Close, 170.15; Vol, 14.20
2017-11-20, Close, 169.98; Vol, 12.78
2017-11-21, Close, 173.14; Vol, 14.54
2017-11-22, Close, 174.96; Vol, 14.69
2017-11-24, Close, 174.97; Vol, 13.84
2017-11-27, Close, 174.09; Vol, 14.08
2017-11-28, Close, 173.07; Vol, 14.96
2017-11-29, Close, 169.48; Vol, 18.18
2017-11-30, Close, 171.85; Vol, 19.06
2017-12-01, Close, 171.05; Vol, 19.40
2017-12-04, Close, 169.80; Vol, 19.91
2